In [51]:

import os
import nltk
import pandas as pd
from datetime import datetime, timedelta
from dotenv import load_dotenv
analyzer = SentimentIntensityAnalyzer()
import alpaca_trade_api as tradeapi
from newsapi.newsapi_client import NewsApiClient
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import hvplot.pandas
%matplotlib inline
# Load .env enviroment variables
load_dotenv()


True

In [52]:
# Reading the News API key enviroment variable
api_key = os.getenv("Api_keys.env")

In [53]:


# Set News API Key
#newsapi = NewsApiClient(api_key=os.environ["news_api"])

# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

#api = tradeapi.REST(alpaca_api_key, alpaca_secret_key, api_version='v2')

In [54]:
load_dotenv(dotenv_path="Api_keys.env")
api_key = os.getenv("news_api")
api_key

'22b1baf9e3d84f549a60cd9ca7881995'

In [55]:
newsapi = NewsApiClient(api_key=api_key)
newsapi

In [59]:
corp_news = newsapi.get_everything(q="AAL", language="en")
corp_news["totalResults"]
corp_df = pd.DataFrame.from_dict(corp_news["articles"])
corp_df.head()

,author,content,description,publishedAt,source,title,url,urlToImage
0,Pete Muntean,Washington (CNN Business)After air travel brok...,After air travel broke a pandemic record on We...,2020-12-25T13:41:29Z,"{'id': 'cnn', 'name': 'CNN'}",Nearly a million people flew in America on Chr...,https://www.cnn.com/2020/12/25/business/tsa-ai...,https://cdn.cnn.com/cnnnext/dam/assets/2012231...
1,Claudia Assis,It was housekeeping day for JPMorgan analysts ...,"United, JetBlue and Spirit shares get a downgr...",2020-12-16T17:53:00Z,"{'id': None, 'name': 'MarketWatch'}",": United, JetBlue stocks get downgrades from J...",https://www.marketwatch.com/story/united-jetbl...,https://images.mktw.net/im-240768/social
2,Tim Dunn,As covid-19 continues to negatively impact the...,The covid-19 financial crisis facing airlines ...,2020-12-17T13:35:35Z,"{'id': None, 'name': 'Seeking Alpha'}",American Airlines' Market Losses Will Aid Comp...,https://seekingalpha.com/article/4395151-ameri...,https://static.seekingalpha.com/uploads/2020/1...
3,Mark Vickery,Non-profit humanitarian agency World Vision Un...,American Airlines (AAL) put its first commerci...,2020-12-29T22:38:10Z,"{'id': None, 'name': 'Yahoo Entertainment'}",Markets Can't Sustain Record Highs; 737 MAX Fl...,https://finance.yahoo.com/news/markets-cant-su...,https://s.yimg.com/uu/api/res/1.2/nYSrGTfdEN.B...
4,Zacks Equity Research,"In the latest trading session, American Airlin...","In the latest trading session, American Airlin...",2020-12-04T22:45:10Z,"{'id': None, 'name': 'Yahoo Entertainment'}",American Airlines (AAL) Outpaces Stock Market ...,https://finance.yahoo.com/news/american-airlin...,https://s.yimg.com/uu/api/res/1.2/1r9UWHxWtEcp...


In [69]:
def get_news(company):
    corp_news = newsapi.get_everything(q=company, language="en")
    corp_news["totalResults"]
    corp_df = pd.DataFrame.from_dict(company_news["articles"])
    corp_sentiments = []

    for article in corp_news["articles"]:
        try:
            text = article["content"]
            date = article["publishedAt"][:10]
            sentiment = analyzer.polarity_scores(text)
            compound = sentiment["compound"]
            pos = sentiment["pos"]
            neu = sentiment["neu"]
            neg = sentiment["neg"]
            
            corp_sentiments.append({
                "text": text,
                "date": date,
                "compound": compound,
                "positive": pos,
                "negative": neg,
                "neutral": neu
                
            })
            
        except AttributeError:
            pass

In [ ]:
# Create DataFrame
company_df = pd.DataFrame(corp_sentiments)

# Reorder DataFrame columns
cols = ["date", "text", "compound", "positive", "negative", "neutral"]
company_df = company_df[cols]
company_df.sort_values(by=['date'], ascending=False, inplace=True)
company_df.reset_index(inplace=True, drop=True)
company_df = company_df.drop(["text", "positive", "negative", "neutral"], axis=1)
company_df = company_df.set_index('date')

return company_df

In [ ]:
companies = ["AAL"]

for company in companies:
    print(company)
    print(get_news(company))